In [1]:

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationBufferMemory(return_messages=True)


def load_memory(_):
    return memory.load_memory_variables({})["history"]

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": RunnableLambda(load_memory),
    }
    | prompt
    | llm
)
questions = ["Is Aaronson guilty?",
             "What message did he write in the table?",
             "Who is Julia?",
]
for question in questions:
    result = chain.invoke(question)
    print(result)
    memory.save_context({"input":question}, {"output":result.content})

content='According to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'
content='He wrote "2+2=5" in the dust on the table.'
content='Julia is a character who was involved with the protagonist in the story.'
